In [ ]:
!pip install -q gdown faiss-gpu

In [ ]:
!gdown --id 1_9NPC8E-kt4pU6rTi5lkh91eFR_jnFav
!unzip -q /kaggle/working/wb_2D3Dretrieval_dataset.zip
!rm /kaggle/working/wb_2D3Dretrieval_dataset.zip

In [ ]:
from logging import currentframe
import numpy as np
from stl import mesh
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os
from matplotlib.colors import Normalize
from scipy.interpolate import griddata
import cv2 as cv
import pandas as pd
from keras.models import Model
from keras import layers, Input
import faiss

In [ ]:
# Define path
path = '/kaggle/working/wb_2D3Dretrieval_dataset'
queries_path = os.path.join(path, 'queries')
database3D_2D_path = os.path.join(path, 'database_2D')


if not os.path.exists(database3D_2D_path):
    os.mkdir(database3D_2D_path)

for obj in os.listdir(os.path.join(path, 'database')):
    obj_name = obj.split('.')[0]

    your_mesh = mesh.Mesh.from_file(os.path.join(path, f'database/{obj}'))

    # Get center of gravity
#     cog = np.mean(your_mesh.vectors.reshape(-1, 3), axis=0)
    cog = [np.max(your_mesh.x) * 0.5, np.max(your_mesh.y) * 0.5, np.max(your_mesh.z) * 0.5]

    # Define the equation of the cutting plane
    normal = np.array([0, 0, 1])  # normal vector of the plane

    intersection_points = []
    for i in range(len(your_mesh.vectors)):
        for j in range(3):
            v0 = your_mesh.vectors[i, j - 1]
            v1 = your_mesh.vectors[i, j]
            if np.dot(normal, v0 - cog) * np.dot(normal, v1 - cog) < 0:
                t = np.dot(normal, cog - v0) / np.dot(normal, v1 - v0)
                intersection_points.append(v0 + t * (v1 - v0))

    # Convert intersection points to NumPy array
    above_plane_vertices = your_mesh.vectors[your_mesh.vectors[:, :, 2] > cog[2]]

    # Flatten the vertices and remove duplicates
    # intersection_points = np.unique(above_plane_vertices.reshape(-1, 3), axis=0)
    intersection_points = above_plane_vertices

    # Convert intersection points to NumPy array
    intersection_points = np.array(intersection_points)

    # Project 3D vertices onto 2D plane (discard z-coordinate)
    projection_points = intersection_points[:, :2]

    # Calculate grayscale values based on z-coordinate
    z_values = intersection_points[:, 2]
    try:
      min_z, max_z = np.min(z_values), np.max(z_values)
    except ValueError as e:
      print(obj)

    # Invert grayscale values
    inverted_values = max_z - z_values

    # Define grid for interpolation
    x_grid = np.linspace(min(projection_points[:, 0]), max(projection_points[:, 0]), 200)
    y_grid = np.linspace(min(projection_points[:, 1]), max(projection_points[:, 1]), 200)
    xx, yy = np.meshgrid(x_grid, y_grid)

    # Perform bilinear interpolation
    interpolated_values = griddata(projection_points, inverted_values, (xx, yy), method='linear')

    # Normalize interpolated values
    norm = Normalize(vmin=min_z, vmax=max_z)
    grayscale_values = norm(interpolated_values)

    plt.figure(figsize=(5.12, 5.12))

    rotated_grayscale_values = np.rot90(grayscale_values, k=2)  # Rotate 180 degrees

    flipped_grayscale_values = np.fliplr(rotated_grayscale_values)  # Horizontal flip

    plt.imshow(flipped_grayscale_values, extent=(min(x_grid), max(x_grid), min(y_grid), max(y_grid)), cmap='gray')
    plt.axis('off')

    plt.savefig(os.path.join(database3D_2D_path, f'/{obj_name}.png'))

    plt.close()

In [ ]:
input_img = keras.Input(shape=(512, 512, 1))

x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
x_train = []
y_train = []

for i in os.listdir(queries_path):
  x_train.append(cv.imread(os.path.join(queries_path, i), cv.IMREAD_GRAYSCALE))

for i in os.listdir(database3D_2D_path):
  y_train.append(cv.imread(os.path.join(database3D_2D_path, i), cv.IMREAD_GRAYSCALE))

x_train = np.array(x_train)
y_train = np.array(y_train)

x_train = x_train.astype('float32') / 255.
y_train = y_train.astype('float32') / 255.

x_train = np.reshape(x_train, (len(x_train), 512, 512, 1))
y_train = np.reshape(y_train, (len(y_train), 512, 512, 1))

x_test = x_train
y_test = y_train

In [ ]:
autoencoder.fit(x_train, y_train,
                epochs=20,
                batch_size=64,
                shuffle=True,
                validation_data=(x_test, y_test))

In [ ]:
decoded_imgs = autoencoder.predict(x_test)

n = 10
plt.figure(figsize=(20, 4))
for i in range(1, n + 1):
    # Display original
    ax = plt.subplot(2, n, i)
    plt.imshow(x_test[i].reshape(512, 512))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + n)
    plt.imshow(decoded_imgs[i].reshape(512, 512))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
def extract_features(image_path, model):
    img = cv.imread(image_path, cv.IMREAD_GRAYSCALE)
    img = np.reshape(img, (1, 512, 512, 1))
    encoder = Model(model.input, model.layers[6].output)
    features = encoder.predict(img)
    return features.flatten()

query_mapping = {}

training_features = []
training_files = []

for file in os.listdir(database3D_2D_path):
    file_path = os.path.join(database3D_2D_path, file)
    features = extract_features(file_path, autoencoder)
    training_features.append(features)
    training_files.append(file)

# Flatten the list of descriptors and create an index
training_features = np.vstack(training_features).astype(np.float32)
index = faiss.IndexFlatL2(training_features.shape[1])
index.add(training_features)

# Function to find top-k matches
def find_top_k_matches(features, k=5):
    features = features.astype(np.float32)
    D, I = index.search(np.expand_dims(features, axis=0), k)
    return I[0]

mrr_at_5 = 0

for query in os.listdir(queries_path):
    query_obj = query.split('.')[0]
    query_path_full = os.path.join(query_path, query)
    query_features = extract_features(query_path_full, autoencoder)

    # Find top-5 matches
    indices = find_top_k_matches(query_features, k=5)

    scores = [(i, training_files[i].split('.')[0] + '.stl') for i in indices]

    query_mapping[query] = ''
    for i in range(5):
        query_mapping[query] += scores[i][1] + ','
        if scores[i][1].split('.')[0] == query_obj:
            mrr_at_5 += 1 / (i + 1)
    query_mapping[query] = query_mapping[query][:-1]

    print(query, query_mapping[query])

queries_len = len(os.listdir(query_path))
print('MRR@5: ', mrr_at_5 / queries_len)